In [18]:
import random
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt

In [24]:
class Team:
    def __init__(self, founder, \
                 foundation_period = -1,
                 a_coef = random.uniform(0, 0.5), \
                    b_coef = random.uniform(0.75, 1.25), \
                        beta_coef = random.uniform(1.5, 2)):
        self.founder = founder
        self.members = {founder} # Используем множество поскольку каждый участник является уникальным
        founder.team = self
        self.a = a_coef
        self.b = b_coef
        self.beta = beta_coef
        self.foundation_period = foundation_period

    def output(self, effort, total_effort_prev):
        return self.a * (total_effort_prev + effort) + \
               self.b * (total_effort_prev + effort) ** self.beta

In [29]:
class Agent:
    def __init__(self, theta, omega, network):
        self.theta = theta
        self.omega = omega
        self.network = network
        self.team = None
        self.effort = 0
        self.utility = 0

    def choose_effort(self, team, total_effort_prev):
        def utility(effort):
            leisure = self.omega - effort
            team_size = len(team.members)
            if team and team != self.team and team_size == 1:
                consumption = \
                    team.output(effort, total_effort_prev) / team_size
            else:
                consumption = \
                    team.output(effort, total_effort_prev - self.effort) / (team_size - 1) if team and len(team.members) > 1 else 0
            return consumption ** self.theta * leisure ** (1 - self.theta)

        def negative_utility(effort):
            return -utility(effort)

        res = minimize(negative_utility, self.omega / 2, bounds = [(0, self.omega)])
        self.effort = res.x[0]
        self.utility = -res.fun

    def choose_team(self, teams, period):
        # Вычислить е* и U(е*) в текущей фирме
        self.choose_effort(self.team, sum(a.effort for a in self.team.members) if self.team else 0)
        current_utility = self.utility
        best_team = self.team
        best_utility = current_utility
        # Вычислить е* и U(е*) во вновь создаваемой фирме
        if len(self.team.members) != 1:
            new_team = Team(self, period)
            self.choose_effort(new_team, 0)
            startup_utility = self.utility
            best_utility = max(current_utility, startup_utility)
            best_team = self.team if current_utility >= startup_utility else new_team

        # ДЛЯ каждой фирмы в социальной сети агента
        for friend in self.network:
            if friend.team and friend.team != self.team:
                # Вычислить е* и U(e*)
                self.choose_effort(friend.team, sum(a.effort for a in friend.team.members))
                if self.utility > best_utility:
                    best_utility = self.utility
                    best_team = friend.team

        # ЕСЛИ текущая фирма - не лучший вариант, покинуть ее
        if self.team != best_team:
            if self.team:
                self.team.members.remove(self)
                if not self.team.members:
                    del teams[self.team]
                    
        # ЕСЛИ стартап - лучший вариант: создать стартап
        if best_team == new_team:
            teams[new_team] = new_team
            best_team.members.add(self)

        # ЕСЛИ другая фирма - лучший вариант: присоединиться
        if best_team != new_team and best_team != self.team:
            best_team.members.add(self)

        self.team = best_team

In [30]:
def print_stats(name, data):
    print(f"{name} - Среднее: {np.mean(data):.2f}, Медиана: {np.median(data):.2f}, Мин: {np.min(data):.2f}, Макс: {np.max(data):.2f}")

def plot_distributions(*data, titles=None):
    num_plots = len(data)
    fig, axs = plt.subplots(2, 3, figsize=(12, 8))
    axs = axs.flatten()

    for i in range(num_plots):
        axs[i].hist(data[i], bins = 20)
        if titles:
            axs[i].set_title(titles[i])
        axs[i].set_xlabel('Значение')
        axs[i].set_ylabel('Частота')

    plt.tight_layout()
    plt.show()

def plot_scatter(x, y, xlabel, ylabel, title):
    plt.figure(figsize = (8, 6))
    plt.scatter(x, y)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.show()

def analyze_period(agents, teams, period, job_changes):
    # Характеристики распределения отдельных показателей
    efforts = [agent.effort for agent in agents]
    output = [agent.team.output(agent.effort, sum(a.effort for a in agent.team.members)) if agent.team else 0 for agent in agents]
    income = [out / len(agent.team.members) if agent.team else 0 for out, agent in zip(output, agents)]
    utility = [agent.utility for agent in agents]
    team_sizes = [len(agent.team.members) if agent.team else 0 for agent in agents]
    team_ages = [period - agent.team.founder.birth_period if agent.team else 0 for agent in agents]

    # Вывод основных характеристик
    print(f"_________________________________________________________________________________")
    print(f"Период: {period}")
    print_stats("Усилия", efforts)
    print_stats("Выпуск", output)
    print_stats("Доход", income)
    print_stats("Полезность", utility)
    print_stats("Размер команды", team_sizes)
    print_stats("Возраст команды", team_ages)

    print(f"Количество агентов: {len(agents)}")
    print(f"Количество команд: {len(teams)}")
    print(f"Средние усилия: {np.mean(efforts):.2f}")
    print(f"Средний выпуск: {np.mean(output):.2f}")
    print(f"Средний доход: {np.mean(income):.2f}")
    print(f"Средняя полезность: {np.mean(utility):.2f}")
    print(f"Количество смен работы: {job_changes}")

    # Гистограммы распределений
    plot_distributions(efforts, output, income, utility, team_sizes, team_ages,
                       titles=["Распределение усилий", "Распределение выпуска", "Распределение дохода",
                               "Распределение полезности", "Распределение размера команды", "Распределение возраста команды"])

    # Взаимосвязь показателей
    plot_scatter(team_ages, team_sizes, "Возраст команды", "Размер команды", "Размер команды vs Возраст команды")

    growth_rates = [len(agent.team.members) / (period - agent.team.founder.birth_period) \
                    if agent.team and period != agent.team.founder.birth_period else 0 for agent in agents]
    plot_scatter(team_ages, growth_rates, "Возраст команды", "Темп роста", "Темп роста vs Возраст команды")

    # Распределения для фирм
    team_output = [sum(agent.effort for agent in team.members) for team in teams.values()]
    team_sizes = [len(team.members) for team in teams.values()]
    team_ages = [period - team.founder.birth_period for team in teams.values()]
    team_productivity = [out / size for out, size in zip(team_output, team_sizes)]
    team_growth_rates = [len(team.members) / (period - team.founder.birth_period) \
                         if period != team.founder.birth_period else 0 for team in teams.values()]

    print(f"\nРаспределения для фирм:")
    print_stats("Выпуск", team_output)
    print_stats("Размер", team_sizes)
    print_stats("Возраст", team_ages)
    print_stats("Производительность", team_productivity)
    print_stats("Темп роста", team_growth_rates)

    # Гистограммы распределений для фирм
    plot_distributions(team_output, team_sizes, team_ages, team_productivity, team_growth_rates,
                       titles=["Распределение выпуска фирм", "Распределение размера фирм", "Распределение возраста фирм",
                               "Распределение производительности фирм", "Распределение темпов роста фирм"])

In [31]:
def simulate(num_agents, num_periods, is_anylize_current_period = False, is_anylize_total = True):
    
    agents = [Agent(random.uniform(0, 1), 1, set()) for _ in range(num_agents)]
    for agent in agents:
        num_friends = random.randint(2, 6)
        agent.network = set(random.sample(agents, num_friends))
    # Создаём словарь где ключ и значение - это команда
    teams = {Team(agent): Team(agent) for agent in agents}

    total_efforts = []
    total_output = []
    total_income = []
    total_utility = []
    job_changes = 0

    for period in range(num_periods):
        active_agents = random.sample(agents, int(num_agents * 0.04))
        for agent in active_agents:
            old_team = agent.team
            agent.choose_team(teams, period)
            if agent.team != old_team:
                job_changes += 1
        # ДЛЯ каждой фирмы
        for team in teams.values():
            # Сложить усилия агентов, произвести продукцию
            total_effort = sum(agent.effort for agent in team.members)
            output = team.output(total_effort, total_effort)

            # Разделить продукцию
            income = output / len(team.members)
            for agent in team.members:
                agent.utility = income ** agent.theta * (agent.omega - agent.effort) ** (1 - agent.theta)
        efforts = [agent.effort for agent in agents]
        output = [agent.team.output(agent.effort, sum(a.effort for a in agent.team.members)) if agent.team else 0 for agent in agents]
        income = [out / len(agent.team.members) if agent.team else 0 for out, agent in zip(output, agents)]
        utility = [agent.utility for agent in agents]

        total_efforts.append(np.mean(efforts))
        total_output.append(np.mean(output))
        total_income.append(np.mean(income))
        total_utility.append(np.mean(utility))

        for team in list(teams.keys()):
            if not team.members:
                del teams[team]

        # Анализ текущего периода
        if is_anylize_current_period:
            analyze_period(agents, teams, period, job_changes)
        
    if is_anylize_total:
        analyze_period(agents, teams, period, job_changes)
    return agents, teams, total_efforts, total_output, total_income, total_utility, job_changes

In [32]:
# Базовая конфигурация
num_agents = 1000
num_periods = 100  # Один модельный период = один календарный месяц
agent_id = 0
team_id = 0
agents, teams, efforts, output, income, utility, job_changes = \
    simulate(num_agents, num_periods, False, True)


set()


KeyError: <__main__.Team object at 0x000002649D560990>

In [48]:
print(f'{len(teams)}')
print(f'{len(agents)}')

9
100
